# Financial News Summarizer + Sentiment Annotator using ChatGPT API

In [39]:
import os
import openai
from dotenv import load_dotenv
import feedparser
from datetime import datetime
import pandas as pd
import time

In [20]:
# STEP 1: Set OpenAI API key
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [21]:
# STEP 2: Parse RSS feed from Yahoo Finance
RSS_FEED_URL = "https://feeds.finance.yahoo.com/rss/2.0/headline?s=^GSPC&region=US&lang=en-US"

def fetch_rss_headlines():
    feed = feedparser.parse(RSS_FEED_URL)
    headlines = [entry.title for entry in feed.entries]
    return headlines

In [35]:
# STEP 3: Query GPT for summary + sentiment

def analyze_headline_with_gpt(headline):
    prompt = (
        f"""
        Summarize the following financial headline in three lines. It should include what the article is about, the conclusion in the article and how it came to the conclusion. 
        Then assess its sentiment as either bullish, bearish, or neutral.

        Headline: {headline}

        Respond in this format exactly:
        Summary: <your two-line summary>
        Sentiment: <bullish/bearish/neutral>
        """
    )

    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            max_tokens=150
        )
        text = response.choices[0].message.content.strip()
        return text
    except Exception as e:
        return f"Error: {e}"

In [36]:
# STEP 4: Parse GPT response (more robust)

def parse_response(response_text):
    print("--- Raw GPT Response ---")
    print(response_text)
    print("------------------------")
    
    summary = ""
    sentiment = ""
    for line in response_text.splitlines():
        if 'summary' in line.lower():
            summary = line.split(":", 1)[-1].strip()
        elif 'sentiment' in line.lower():
            sentiment = line.split(":", 1)[-1].strip().lower()
    return summary, sentiment

In [37]:
# STEP 5: Process workflow

def run_pipeline():
    headlines = fetch_rss_headlines()
    results = []

    for headline in headlines:
        gpt_response = analyze_headline_with_gpt(headline)
        summary, sentiment = parse_response(gpt_response)
        results.append({
            "date": datetime.now().strftime('%Y-%m-%d'),
            "headline": headline,
            "summary": summary,
            "sentiment": sentiment
        })
        time.sleep(1.5)  # Respect API rate limits

    df = pd.DataFrame(results)
    df.to_csv("financial_news_summary.csv", index=False)
    print(df.head())

In [38]:
if __name__ == "__main__":
    run_pipeline()

--- Raw GPT Response ---
Summary: The article explains what a P/E ratio is and how it can be used to evaluate investments, providing a valuable tool for investors to make informed decisions.
Sentiment: Neutral
------------------------
--- Raw GPT Response ---
Summary: The article discusses the potential profitability of investing in Pfizer Inc. as a cheap stock option. It concludes that Pfizer Inc. may be a good investment opportunity based on its current valuation and growth prospects.
Sentiment: Bullish
------------------------
--- Raw GPT Response ---
Summary: The article discusses whether Wells Fargo & Company is a profitable cheap stock to buy at the moment. It concludes that WFC may be a good investment opportunity due to its potential for profitability.
Sentiment: Bullish
------------------------
--- Raw GPT Response ---
Summary: The article discusses whether Johnson & Johnson (JNJ) is a profitable cheap stock to buy currently. It concludes that JNJ is indeed a good investment o